In [29]:
from itertools import combinations
import numpy as np

In [31]:
def calculate_objective(
    solution_encoding: np.array, flow: np.array, distance: np.array
) -> int:
    cost = 0

    for object_i, location_i in enumerate(solution_encoding):
        for object_j, location_j in enumerate(solution_encoding):
            cost += flow[object_i, object_j] * distance[location_i, location_j]

    return cost

In [32]:
def adjacent_swap_generator(solution_encoding: list, n: int):
    idx = 0
    neighbour = solution_encoding.copy()
    while idx + 1 <= n:
        neighbour[idx], neighbour[(idx + 1) % n] = neighbour[(idx + 1) % n], neighbour[idx]
        yield neighbour
        neighbour[(idx + 1) % n], neighbour[idx] = neighbour[idx], neighbour[(idx + 1) % n]
        idx += 1

def generate_one_adjacent_swap_neighbourhood(solution_encoding: list, n: int) -> list:
    neighbourhood = []
    for idx, _ in enumerate(solution_encoding):
        neighbour = solution_encoding.copy()
        neighbour[idx], neighbour[(idx + 1) % n] = neighbour[(idx + 1) % n], neighbour[idx]
        neighbourhood.append(neighbour)
    return neighbourhood

In [34]:
for _ in range(10000):   
    x = adjacent_swap_generator(list(range(0, 256)), 256)
    for encoding in x:
        break

In [35]:
def incremental_calculate_objective(current_encoding: np.array, n: int,
                                    current_objective: int, swap_index: tuple,
                                    flow: np.array, distance: np.array) -> int:
    cost = current_objective
    swap_dict = {swap_index[0] : swap_index[1], swap_index[1] : swap_index[0]}
    for i in range(n):
        if i in swap_index:
            for j in range(n):
                if not j in swap_index:
                    cost -= flow[i, j] * distance[current_encoding[i], current_encoding[j]]
                    
                    cost += flow[i, j] * distance[current_encoding[swap_dict[i]], current_encoding[j]]
                else:
                    cost -= flow[i, j] * distance[current_encoding[i], current_encoding[j]]
                    
                    cost += flow[i, j] * distance[current_encoding[swap_dict[i]], current_encoding[swap_dict[j]]]
        else:
            cost -= flow[i, swap_index[0]] * distance[current_encoding[i], current_encoding[swap_index[0]]]
            cost -= flow[i, swap_index[1]] * distance[current_encoding[i], current_encoding[swap_index[1]]]
            
            cost += flow[i, swap_index[0]] * distance[current_encoding[i], current_encoding[swap_index[1]]]
            cost += flow[i, swap_index[1]] * distance[current_encoding[i], current_encoding[swap_index[0]]]

    return cost


In [36]:
flow = np.array([[1,9,3], [4,5,6], [7,8,9]])
distance = np.array([[1,2,3], [4,5,6], [7,8,9]])

In [37]:
calculate_objective([0,1,2], flow, distance)

299

In [38]:
calculate_objective([1,0,2], flow, distance)

283

In [40]:
incremental_calculate_objective([0,1,2], 3, 299, (0, 1), flow, distance)

283